In [2]:
! pip install datasets

  Using cached datasets-2.13.1-py3-none-any.whl (486 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.9/336.9 kB 778.2 kB/s eta 0:00:001m819.2 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.6/22.6 MB 4.9 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 3.9 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)


In [67]:
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

In [29]:
from datasets import load_dataset, Dataset

#dataset = load_dataset("financial_phrasebank")
df=load_dataset('financial_phrasebank', 'sentences_allagree')

Found cached dataset financial_phrasebank (/Users/mohamedyoussfi/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)
100%|██████████| 1/1 [00:00<00:00, 805.67it/s]


In [32]:
df['train']['sentence']

['According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
 "For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .",
 'In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .',
 'Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales .',
 'Operating profit totalled EUR 21.1 mn , up from EUR 18.6 mn in 2007 , representing 9.7 % of net sales .',
 'Finnish Talentum reports its operating profit increased to EUR 20.5 mn in 2005 from EUR 9.3 mn in 2004 , and net sales totaled EUR 103.3 mn , up from EUR 96.4 mn .',
 "Clothing retail chain Sepp+ñl+ñ 's sales increased by 8 % to EUR 155.2 mn , and operating profit rose to EUR 31.1 mn from EUR 17.1 mn i

In [7]:
import pandas as pd

In [76]:
df1=pd.DataFrame()

In [77]:
df1['sentence']=df.data['train']['sentence']
df1['label']=df.data['train']['label']

In [74]:
#df1['output']=df1['label'].apply(lambda x:[1,0,0] if x==2 else [0,1,0] if x==0 else [0,0,1])
#encoder = LabelEncoder()
#encoder.fit(df1['label'])
#encoded_Y = encoder.transform(df1['label'])
#df1['output']=np_utils.to_categorical(encoded_Y,3)

array([1, 2, 2, ..., 0, 0, 0])

In [75]:
df1.sample(10)

,sentence,label,output
634,The writing and publication of Lemmink+ñinen -...,1,0.0
1910,"TietoEnator was down 1.13 pct to 18.38 , exten...",0,1.0
1207,"Deliveries of Nokia 1112 , Nokia 2310 and Noki...",1,0.0
860,"Cargotec Corporation , Press Release , August ...",2,0.0
479,"Turun kaupunkin , Finland based company has aw...",2,0.0
986,"Its customers include local companies Slo Oy ,...",1,0.0
1868,Finnish media group Talentum has issued a prof...,0,1.0
1786,The firm builds components for mobile phones a...,1,0.0
976,He confirmed his view on July 6 .,1,0.0
2263,Sales in Finland decreased by 10.5 % in Januar...,0,1.0


In [42]:
df1['label'].value_counts()
positive_sentences=df1[df1['label']==2]
neutral_sentences=df1[df1['label']==1]
negative_sentences=df1[df1['label']==0]

In [45]:
bert_encoder_url="https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
bert_preprocess_url="https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
#bert_preprocess_model=hub.KerasLayer(bert_preprocess_url)
bert_preprocess_model=hub.KerasLayer('bert_en_uncased_preprocess_3')
#bert_encoder_model=hub.KerasLayer(bert_encoder_url)
bert_encoder_model=hub.KerasLayer('bert_en_uncased_L-12_H-768_A-12_4')

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-07-08 18:45:04.066485: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [47]:
input_layer = tf.keras.layers.Input(shape=(), name="input", dtype=tf.dtypes.string)
preprocess_layer=bert_preprocess_model(input_layer)
encoder_layer = bert_encoder_model(preprocess_layer)
dropout_layer = tf.keras.layers.Dropout(rate=0.1, name ='dropout')(encoder_layer['pooled_output'])
output_layer = tf.keras.layers.Dense(3, activation='softmax', name = 'output')(dropout_layer)
model = tf.keras.Model(inputs = [input_layer], outputs=[output_layer])

In [48]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['input[0][0]']                  
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [94]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])

In [88]:
from sklearn.model_selection import train_test_split

In [89]:
x_train, x_test, y_train, y_test = train_test_split(
    df1['sentence'], 
    df1['label'],
    stratify=df1['label'],
    test_size=0.2
    )

In [90]:
y_train.info()

<class 'pandas.core.series.Series'>
Index: 1811 entries, 1082 to 1410
Series name: label
Non-Null Count  Dtype
--------------  -----
1811 non-null   int64
dtypes: int64(1)
memory usage: 28.3 KB


In [91]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)

In [92]:
y_train

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [100]:
tf.config.run_functions_eagerly(True)
history=model.fit(x_train,y_train, epochs=10, validation_data=[x_test])

Epoch 1/10


/Users/mohamedyoussfi/miniconda3/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


57/57 [==============================] - 833s 15s/step - loss: 0.7536 - accuracy: 0.6814 - val_loss: 0.6886 - val_accuracy: 0.6584
Epoch 2/10
57/57 [==============================] - 823s 14s/step - loss: 0.7006 - accuracy: 0.7035 - val_loss: 0.6436 - val_accuracy: 0.7355
Epoch 3/10
45/57 [======================>.......] - ETA: 2:58:41 - loss: 0.6866 - accuracy: 0.7201

KeyboardInterrupt: 